In [11]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import numpy as np
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import json
from collections import Counter

In [12]:
# データ読み込み
molecule_db = pd.read_csv('../datas/molecules.csv', names=['id', 'none', 'pubchem_id', 'common_name', 'flavor_profile'], header=0)
flavor_db = pd.read_csv('../datas/flavor_db.csv', names=['id', 'entry_id', 'alias', 'synonyms', 'scientific_name', 'category', 'molecules'], header=0)

flavor_db.drop(['id'], axis=1, inplace=True)
flavor_db.tail()

,,entry_id,alias,synonyms,scientific_name,category,molecules
935,935.0,0,egg,{'Egg'},Chicken,Animal Product,"{6274, 5311110, 644104, 9609, 18827, 527, 1960..."
936,936.0,979,olive oil,{''},Olea europaea L.,additive,"{6184, 31260, 5281168, 8103}"
937,NaN,980,Baking powder,NaN,NaN,NaN,"{11265, 62465, 644104, 12297, 31242, 527, 4114..."
938,NaN,981,Baking soda,NaN,NaN,NaN,{10340}
939,NaN,982,alum,NaN,NaN,NaN,{24856}


In [13]:
molecule_db.tail(3)

,id,none,pubchem_id,common_name,flavor_profile
1788,1788,NaN,10340,Sodium Carbonate,odorless
1789,1789,NaN,24856,Potassium alum,odorless
1790,1790,NaN,24403,Tetrasodium Pyrophosphate,odorless


In [14]:

result_array = []
all_flavors = molecule_db['flavor_profile'].apply(lambda x: x.replace("'", "").strip('{}').split(', ')).tolist()
for row in all_flavors:
    new_row = []
    for flavor in row:
        if flavor != '':
            new_row.append(flavor)
    result_array.append(new_row)
        
for i, row in flavor_db.iterrows():
    molecules = str(row['molecules']).replace("'", "").strip('{}').split(', ')
    synonyms = str(row['synonyms']).replace("'", "").strip('{}').split(', ')
    
    result_array.append([row['alias'], 'means'] + synonyms)
    result_array.append([row['alias'], 'means', row['scientific_name']])
    for mol_id in molecules:        
        str_flavors = molecule_db[molecule_db['id'] == int(mol_id)]['flavor_profile']
        flavor_list = str_flavors.apply(lambda x: x.replace("'", "").strip('{}').split(', ')).tolist()
        if len(flavor_list) > 0 and flavor_list[0] != ['']:
            for flavor in flavor_list[0]:
                result_array.append([row['alias'], 'has', 'flavor', 'of', flavor])

In [18]:

# NumPyの配列に変換
np_sentences = np.array(result_array, dtype=object)

# NumPyの配列をdumpして保存
np.save('../datas/flavor_sentences.npy', np_sentences)
np.savetxt('../datas/flavor_sentences.txt', np_sentences, fmt="%s")